In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Prueba de hipótesis

Para determinar qué prueba de hipótesis realizar con este conjunto de datos, primero debemos identificar la pregunta o la afirmación que deseamos investigar. Aquí planteamos algunos ejemplos de preguntas o afirmaciones que probaremos para entender en que consiste la prueba de hipótesis:

1. **Hipótesis sobre los ingresos promedio en función de la educación:**
   - Hipótesis nula (H0): No hay diferencia en los ingresos promedio entre las personas con educación básica 4 años y las personas con educación secundaria.
   - Hipótesis alternativa (H1): Hay una diferencia significativa en los ingresos promedio entre estos dos grupos.
   - Prueba de hipótesis: Se podría realizar una prueba t de Student para comparar los ingresos promedio de estos dos grupos.

2. **Hipótesis sobre la duración de la llamada telefónica en función del estado civil:**
   - Hipótesis nula (H0): No hay diferencia en la duración promedio de las llamadas telefónicas entre personas casadas y personas solteras.
   - Hipótesis alternativa (H1): Hay una diferencia significativa en la duración promedio de las llamadas telefónicas entre estos dos grupos.
   - Prueba de hipótesis: También se podría utilizar una prueba t de Student para comparar la duración promedio de las llamadas entre los dos grupos de estado civil.

Para poder enfrentarnos a estos problemas vamos a tener que usar el método `ttest_ind()` de la librería `stats`.  Lo que hace este método es usar el "Test de Student".

La función `ttest_ind` se utiliza para realizar una prueba t de Student independiente entre dos muestras (grupos) para comparar si sus medias son estadísticamente diferentes. Aquí está la sintaxis básica:

```python
ttest_ind(a, b, axis, equal_var, nan_policy=)
```


Uno de los parámetros que tenemos en el `ttest_ind` es el `equal_var`, donde le tenemos que decir, si nuestras categorías a comparar tienen varianzas iguales o no. Por lo tanto, lo primero que tendremos que hacer es evaluarlo, para eso tenemos dos test: **Prueba de Levene**
y la **Prueba de Bartlett**. 

> En ambos casos, si el p-valor < 0.05 podemos concluir que las varianzas son diferentes entre los grupos. Si el p-valor > 0.05, no podemos afirmar que las varianzas son diferentes.

In [9]:
# lo primero que hacemos es cargar el dataframe que vamos a usar para esta lección
df = pd.read_csv("../files/bank-additional_clean.csv", index_col = 0)
df.head(2)

,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,y,date,latitude,longitude,contact_month,contact_year,age_cat,hijos_totales
0,161770.0,1.0,0.0,2012-04-04,29.0,089b39d8-e4d0-461b-87d4-814d71e0e079,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no,2-agosto-2019,41.495,-71.233,agosto,2019.0,Adultos mayores,1.0 hijos
1,85477.0,1.0,1.0,2012-12-30,7.0,e9d37224-cb6f-4942-98d7-46672963d097,57.0,services,married,high school,NaN,No,No,telephone,149,1,NaN,0,nonexistent,1.1,93.994,-36.4,NaN,5191,no,14-septiembre-2016,34.601,-83.923,septiembre,2016.0,Mediana edad,2.0 hijos


In [11]:
# vamos a definir una función para poder hacer el t-test de Student

def prueba_hipotesis(*args):
    
    # lo primero que tenemos que hacer es mirar si las varianzas son iguales o no
    if len(args) == 2:
        p_valor_varianza = stats.levene(*args, center = "median")[1]
    else:
        p_valor_varianza = stats.bartlett(*args)[1]
    
    if p_valor_varianza > 0.05:
        # realizamos la prueba t de Student
        t_stat, p_valor = stats.ttest_ind(*args, equal_var=True)
    else:
        t_stat, p_valor = stats.ttest_ind(*args, equal_var=False)
        
    # Establecemos un nivel de significancia (alfa)
    alfa = 0.05

    # comparamos el p-valor con el nivel de significancia
    if p_valor < alfa:
        print("Rechazamos la hipótesis nula.")
        print("Hay una diferencia significativa en los ingresos promedio entre los dos grupos.")
    else:
        print("No podemos rechazar la hipótesis nula.")
        print("No hay evidencia suficiente para afirmar una diferencia significativa en los ingresos promedio entre los dos grupos.")

In [12]:

# filtramos los datos para obtener dos grupos: educación básica 4 años y educación secundaria
grupo_educacion_basica = df[df['education'] == 'basic 4y']["income"]
grupo_educacion_secundaria = df[df['education'] == 'high school']["income"]

# llamamos a la función que hemos creado
prueba_hipotesis(grupo_educacion_basica, grupo_educacion_secundaria )


Rechazamos la hipótesis nula.
Hay una diferencia significativa en los ingresos promedio entre los dos grupos.


In [13]:
# vamos a probar la segunda hipótesis que tenemos planteada
# No hay diferencia en la duración promedio de las llamadas telefónicas entre personas casadas y personas solteras.

# lo primero que tenemos que hacer es crear dos DataFrames nuevos, uno para saber el tiempo de las llamadas a personas casadas y otro para personas solteras
grupo_casadas = df[df["marital"] == "married"]["duration"]
grupo_solteras = df[df["marital"] == "single"]["duration"]

# llamamos a la función que hemos creado
prueba_hipotesis(grupo_casadas, grupo_solteras )

No podemos rechazar la hipótesis nula.
No hay evidencia suficiente para afirmar una diferencia significativa en los ingresos promedio entre los dos grupos.
